# Exercices : Chiffrement Symétrique

**Objectifs** :
- Vérifier la compréhension des modes AES
- Attaquer des implémentations faibles
- Tester la sécurité CPA
- Comprendre l'importance des IV/nonces

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import secrets
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

## Exercice 1 : Détection de mode ECB

**Contexte** : Un serveur chiffre des données avec AES mais vous ne savez pas quel mode est utilisé.

**Tâche** : Écrivez une fonction qui détecte si le mode est ECB en analysant les ciphertexts.

In [ ]:
class BlackBoxCipher:
    """
    Oracle de chiffrement dont le mode est inconnu.
    """
    def __init__(self, mode='ecb'):
        self.key = secrets.token_bytes(16)
        self.mode = mode  # 'ecb' ou 'cbc' (secret)
    
    def encrypt(self, plaintext: bytes) -> bytes:
        """
        Chiffre le plaintext avec un mode inconnu.
        """
        # Padding
        padder = padding.PKCS7(128).padder()
        padded = padder.update(plaintext) + padder.finalize()
        
        if self.mode == 'ecb':
            cipher = Cipher(algorithms.AES(self.key), modes.ECB(), backend=default_backend())
        else:  # cbc
            iv = secrets.token_bytes(16)
            cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv), backend=default_backend())
        
        encryptor = cipher.encryptor()
        return encryptor.update(padded) + encryptor.finalize()

def detect_ecb_mode(oracle) -> bool:
    """
    Détecte si l'oracle utilise le mode ECB.
    
    Indice : Envoyez un message avec des blocs répétitifs.
    
    Returns:
        True si ECB détecté, False sinon
    """
    # À COMPLÉTER
    # Stratégie : Envoyer un message avec blocs identiques
    # Si ECB : blocs ciphertext identiques
    # Si CBC : blocs ciphertext différents (IV aléatoire)
    
    message = b'A' * 64  # 4 blocs identiques
    ciphertext = oracle.encrypt(message)
    
    # Découper en blocs de 16 bytes
    blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    
    # Vérifier doublons
    return len(blocks) != len(set(blocks))

# Test
print("=" * 60)
print("TEST : Détection de mode ECB")
print("=" * 60)

oracle_ecb = BlackBoxCipher(mode='ecb')
oracle_cbc = BlackBoxCipher(mode='cbc')

is_ecb_1 = detect_ecb_mode(oracle_ecb)
is_ecb_2 = detect_ecb_mode(oracle_cbc)

print(f"\nOracle 1 (ECB) détecté comme ECB : {is_ecb_1} {'✅' if is_ecb_1 else '❌'}")
print(f"Oracle 2 (CBC) détecté comme ECB : {is_ecb_2} {'❌ (correct)' if not is_ecb_2 else '✅ (erreur)'}")

## Exercice 2 : Attaque sur IV prévisible (CBC)

**Contexte** : Un serveur utilise CBC avec un IV prévisible : $IV_i = E_k(i)$ où $i$ est un compteur.

**Tâche** : Démontrez que ce système n'est pas CPA-sécurisé.

In [ ]:
class PredictableIVOracle:
    """
    Oracle CBC avec IV prévisible (VULNÉRABLE).
    """
    def __init__(self):
        self.key = secrets.token_bytes(16)
        self.counter = 0
    
    def _get_next_iv(self) -> bytes:
        """
        Génère IV prévisible : IV = AES_k(counter).
        """
        cipher = Cipher(algorithms.AES(self.key), modes.ECB(), backend=default_backend())
        encryptor = cipher.encryptor()
        iv = encryptor.update(self.counter.to_bytes(16, 'big')) + encryptor.finalize()
        self.counter += 1
        return iv
    
    def encrypt(self, plaintext: bytes) -> tuple[bytes, bytes]:
        """
        Chiffre avec CBC et IV prévisible.
        
        Returns:
            (IV, ciphertext)
        """
        iv = self._get_next_iv()
        
        # Padding
        padder = padding.PKCS7(128).padder()
        padded = padder.update(plaintext) + padder.finalize()
        
        cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(padded) + encryptor.finalize()
        
        return iv, ciphertext
    
    def peek_next_iv(self) -> bytes:
        """
        Permet à l'attaquant de prédire le prochain IV.
        """
        cipher = Cipher(algorithms.AES(self.key), modes.ECB(), backend=default_backend())
        encryptor = cipher.encryptor()
        return encryptor.update(self.counter.to_bytes(16, 'big')) + encryptor.finalize()

def cpa_attack_predictable_iv():
    """
    Attaque CPA sur CBC avec IV prévisible.
    
    Stratégie :
    1. Choisir deux messages m0, m1
    2. Prédire IV prochain : IV*
    3. Envoyer m0' = m0 ⊕ IV* et m1' = m1 ⊕ IV*
    4. Observer premier bloc de ciphertext
    5. Distinguer car c[0] = E_k(m_b' ⊕ IV*) = E_k(m_b)
    """
    oracle = PredictableIVOracle()
    
    print("=" * 60)
    print("ATTAQUE CPA : CBC avec IV prévisible")
    print("=" * 60)
    
    # Messages challenge
    m0 = b'A' * 16
    m1 = b'B' * 16
    
    print(f"\nMessages challenge :")
    print(f"  m0 : {m0}")
    print(f"  m1 : {m1}")
    
    # Prédire le prochain IV
    predicted_iv = oracle.peek_next_iv()
    print(f"\n🔮 IV prédit : {predicted_iv.hex()[:32]}...")
    
    # Créer messages modifiés : m' = m ⊕ IV_predicted ⊕ 0
    # Pour que E_k(m' ⊕ IV_predicted) = E_k(m ⊕ 0) = E_k(m)
    m0_modified = bytes(a ^ b for a, b in zip(m0, predicted_iv))
    m1_modified = bytes(a ^ b for a, b in zip(m1, predicted_iv))
    
    # L'oracle choisit m_b (on simule b=0)
    b = 0  # Secret choisi par l'oracle
    mb = m0 if b == 0 else m1
    mb_modified = m0_modified if b == 0 else m1_modified
    
    # Chiffrement
    iv_actual, ciphertext = oracle.encrypt(mb_modified)
    
    print(f"\n📤 Message chiffré (m{b}')")
    print(f"  IV utilisé  : {iv_actual.hex()[:32]}...")
    print(f"  IV prédit ? : {iv_actual == predicted_iv} ✅")
    
    # Comparer avec chiffrements directs
    # On chiffre m0 et m1 avec IV=0 pour obtenir références
    cipher_ref = Cipher(algorithms.AES(oracle.key), modes.ECB(), backend=default_backend())
    encryptor_ref = cipher_ref.encryptor()
    
    c0_expected = encryptor_ref.update(m0)
    c1_expected = encryptor_ref.update(m1)
    
    first_block = ciphertext[:16]
    
    print(f"\n🔍 Analyse du premier bloc :")
    print(f"  Bloc reçu     : {first_block.hex()}")
    print(f"  E_k(m0) attendu : {c0_expected.hex()}")
    print(f"  E_k(m1) attendu : {c1_expected.hex()}")
    
    # Distinguer
    guess = 0 if first_block == c0_expected else 1
    
    print(f"\n🎯 Attaquant devine : b = {guess}")
    print(f"   Valeur réelle     : b = {b}")
    print(f"   {'✅ SUCCÈS' if guess == b else '❌ ÉCHEC'}")
    
    print(f"\n⚠️  CBC avec IV prévisible n'est PAS CPA-sécurisé !")
    print(f"✅ Solution : Utiliser IV aléatoire et imprévisible")

cpa_attack_predictable_iv()

## Exercice 3 : Nonce Reuse en CTR

**Contexte** : Deux messages ont été chiffrés avec AES-CTR et le même nonce.

**Tâche** : Retrouvez les messages à partir des ciphertexts.

In [ ]:
def ctr_nonce_reuse_attack():
    """
    Démonstration de l'attaque par réutilisation de nonce en CTR.
    """
    print("=" * 60)
    print("ATTAQUE : Réutilisation de nonce en CTR")
    print("=" * 60)
    
    key = secrets.token_bytes(16)
    nonce = secrets.token_bytes(16)  # Nonce réutilisé !
    
    # Deux messages secrets
    m1 = b"Attack at dawn tomorrow"
    m2 = b"Retreat immediately now"
    
    print(f"\n🔒 Messages secrets (inconnus de l'attaquant) :")
    print(f"  m1 : {m1}")
    print(f"  m2 : {m2}")
    
    # Chiffrement CTR avec MÊME nonce
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
    encryptor1 = cipher.encryptor()
    c1 = encryptor1.update(m1) + encryptor1.finalize()
    
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
    encryptor2 = cipher.encryptor()
    c2 = encryptor2.update(m2) + encryptor2.finalize()
    
    print(f"\n📡 Ciphertexts interceptés :")
    print(f"  c1 : {c1.hex()}")
    print(f"  c2 : {c2.hex()}")
    
    # Attaque : c1 ⊕ c2 = m1 ⊕ m2 (le keystream s'annule !)
    xor_result = bytes(a ^ b for a, b in zip(c1, c2))
    
    print(f"\n💥 c1 ⊕ c2 = m1 ⊕ m2 :")
    print(f"  {xor_result.hex()}")
    print(f"  (ASCII) : {xor_result}")
    
    # Si l'attaquant connaît m1 (ou peut le deviner), il récupère m2 !
    print(f"\n🔓 Si l'attaquant devine/connaît m1 :")
    m2_recovered = bytes(a ^ b for a, b in zip(xor_result, m1))
    print(f"  m2 récupéré : {m2_recovered}")
    print(f"  m2 original : {m2}")
    print(f"  {'✅ IDENTIQUE !' if m2_recovered == m2 else '❌'}")
    
    print(f"\n⚠️  CATASTROPHE : Nonce reuse en CTR = Two-Time Pad !")
    print(f"✅ Solution : TOUJOURS utiliser un nonce unique par message")

ctr_nonce_reuse_attack()

## Exercice 4 : Test de distribution PRG

**Contexte** : Vérifier qu'un générateur produit une sortie indistinguable du random.

**Tâche** : Implémentez des tests statistiques (fréquence, runs test).

In [ ]:
def frequency_test(bits: str, alpha=0.01) -> bool:
    """
    Test de fréquence (nombre de 0 vs 1).
    
    H0 : La séquence est aléatoire
    
    Args:
        bits: Chaîne de bits ('010101...')
        alpha: Seuil de significativité
    
    Returns:
        True si le test passe (semble aléatoire)
    """
    n = len(bits)
    ones = bits.count('1')
    
    # Statistique de test
    s = abs(ones - n/2) / np.sqrt(n/4)
    
    # Valeur critique pour alpha=0.01 (distribution normale)
    critical = 2.576  # pour alpha=0.01 bilatéral
    
    return s < critical

def runs_test(bits: str, alpha=0.01) -> bool:
    """
    Test des runs (longueurs de séquences de bits identiques).
    
    Un "run" est une séquence de bits identiques (ex: '111' ou '00').
    
    Args:
        bits: Chaîne de bits
        alpha: Seuil de significativité
    
    Returns:
        True si le test passe
    """
    n = len(bits)
    ones = bits.count('1')
    pi = ones / n
    
    # Pré-test : fréquence de 1 doit être proche de 0.5
    if abs(pi - 0.5) >= 2/np.sqrt(n):
        return False
    
    # Compter les runs
    runs = 1
    for i in range(1, n):
        if bits[i] != bits[i-1]:
            runs += 1
    
    # Statistique de test
    expected_runs = 2 * n * pi * (1 - pi)
    variance_runs = 2 * n * pi * (1 - pi) * (2 * n * pi * (1 - pi) - 1)
    
    if variance_runs <= 0:
        return False
    
    z = (runs - expected_runs) / np.sqrt(variance_runs)
    
    critical = 2.576
    return abs(z) < critical

# Test sur vrai random vs PRG
print("=" * 60)
print("TESTS STATISTIQUES : Random vs PRG")
print("=" * 60)

# Vrai random
true_random_bytes = secrets.token_bytes(1000)
true_random_bits = ''.join(format(b, '08b') for b in true_random_bytes)

# PRG (ChaCha20)
key = secrets.token_bytes(32)
nonce = secrets.token_bytes(16)
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
encryptor = cipher.encryptor()
prg_bytes = encryptor.update(b'\x00' * 1000)  # Chiffrer des zéros
prg_bits = ''.join(format(b, '08b') for b in prg_bytes)

# Faux random (pattern)
fake_bits = '01' * 4000  # Pattern évident

print(f"\nTaille des séquences : {len(true_random_bits)} bits\n")

# Test vrai random
freq_pass = frequency_test(true_random_bits)
runs_pass = runs_test(true_random_bits)
print(f"Vrai Random (secrets.token_bytes) :")
print(f"  Frequency test : {'✅ PASS' if freq_pass else '❌ FAIL'}")
print(f"  Runs test      : {'✅ PASS' if runs_pass else '❌ FAIL'}")

# Test PRG
freq_pass = frequency_test(prg_bits)
runs_pass = runs_test(prg_bits)
print(f"\nPRG (ChaCha20) :")
print(f"  Frequency test : {'✅ PASS' if freq_pass else '❌ FAIL'}")
print(f"  Runs test      : {'✅ PASS' if runs_pass else '❌ FAIL'}")

# Test faux random
freq_pass = frequency_test(fake_bits)
runs_pass = runs_test(fake_bits)
print(f"\nFaux Random (pattern '01010101...') :")
print(f"  Frequency test : {'✅ PASS' if freq_pass else '❌ FAIL'}")
print(f"  Runs test      : {'✅ PASS' if runs_pass else '❌ FAIL'}")

print(f"\n💡 Un bon PRG doit passer les tests statistiques standard")
print(f"   (mais passer les tests ne garantit PAS la sécurité cryptographique !)")

## Exercice 5 : Padding Oracle Preliminaire

**Contexte** : Introduction au problème du padding (sera détaillé au Chapitre 3).

**Tâche** : Implémentez et testez le padding PKCS#7.

In [ ]:
def pkcs7_pad(data: bytes, block_size: int = 16) -> bytes:
    """
    Applique le padding PKCS#7.
    
    Règle : Ajouter N bytes de valeur N, où N = block_size - (len(data) % block_size)
    Si len(data) est multiple de block_size, ajouter un bloc complet de padding.
    
    Exemples (block_size=16) :
    - "ABC" (3 bytes) → "ABC\x0d\x0d..\x0d" (13 fois \x0d)
    - "ABCDEFGHIJKLMNOP" (16 bytes) → "ABCDEFGHIJKLMNOP\x10\x10..\x10" (16 fois \x10)
    """
    padding_length = block_size - (len(data) % block_size)
    padding = bytes([padding_length] * padding_length)
    return data + padding

def pkcs7_unpad(padded_data: bytes, block_size: int = 16) -> bytes:
    """
    Retire le padding PKCS#7.
    
    Raises:
        ValueError: Si le padding est invalide
    """
    if len(padded_data) == 0 or len(padded_data) % block_size != 0:
        raise ValueError("Invalid padded data length")
    
    padding_length = padded_data[-1]
    
    if padding_length == 0 or padding_length > block_size:
        raise ValueError("Invalid padding length")
    
    # Vérifier que tous les bytes de padding sont corrects
    for i in range(padding_length):
        if padded_data[-(i+1)] != padding_length:
            raise ValueError("Invalid padding bytes")
    
    return padded_data[:-padding_length]

# Tests
print("=" * 60)
print("PADDING PKCS#7")
print("=" * 60)

test_cases = [
    b"ABC",
    b"ABCDEFGHIJKLMNO",   # 15 bytes
    b"ABCDEFGHIJKLMNOP",  # 16 bytes (multiple)
    b"Hello, World!",
    b"",  # Cas vide
]

for data in test_cases:
    padded = pkcs7_pad(data, 16)
    unpadded = pkcs7_unpad(padded, 16)
    
    print(f"\nDonnées originales : {data} ({len(data)} bytes)")
    print(f"Après padding      : {padded.hex()} ({len(padded)} bytes)")
    print(f"Après unpadding    : {unpadded}")
    print(f"Vérification       : {'✅' if unpadded == data else '❌'}")

# Test padding invalide
print("\n" + "=" * 60)
print("TEST : Padding invalide")
print("=" * 60)

invalid_paddings = [
    b"ABC\x05\x05\x05\x05",  # Bon
    b"ABC\x05\x05\x05\x04",  # Mauvais (dernier byte différent)
    b"ABC\x00\x00\x00\x00",  # Padding de 0 (invalide)
    b"ABC\x11\x11\x11\x11",  # Padding > block_size
]

for padded in invalid_paddings:
    try:
        result = pkcs7_unpad(padded, 16)
        print(f"\nPadding : {padded.hex()}")
        print(f"Résultat : {result} ✅")
    except ValueError as e:
        print(f"\nPadding : {padded.hex()}")
        print(f"Erreur  : {e} ❌ (attendu)")

print(f"\n💡 Le padding oracle attack exploite ces erreurs de validation !")
print(f"   (Détails au Chapitre 3)")

## Conclusion

**Points clés vérifiés** :
- ❌ ECB est détectable par analyse de répétitions
- ❌ IV prévisible en CBC casse la sécurité CPA
- ❌ Nonce reuse en CTR = catastrophe (Two-Time Pad)
- ✅ PRG cryptographiques passent tests statistiques standard
- ✅ Padding PKCS#7 doit être validé correctement

**Retenir** :
- Toujours utiliser IV/nonce aléatoires et uniques
- Ne JAMAIS utiliser ECB
- Privilégier CTR ou modes AEAD (Chapitre 3)
- Valider soigneusement le padding